In [1]:
import math
import random

In [2]:
# pip install graphviz
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [15]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __radd__(self, other): # other + self
        return self + other
        
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __truediv__(self, other):
        return self * other**-1

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data**other, (self, ), 'pow')

        def _backward():
            self.grad += other * self.data**(other - 1) * out.grad
        out._backward = _backward

        return out

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __sub__(self, other): # self - other
        return self + (-other)

    def __neg__(self): # -self
        return self * -1

    def exp(self):
        x = math.exp(self.data)
        out = Value(x, (self, ), 'exp')

        def _backward():
            self.grad += x * out.grad
        out._backward = _backward

        return out

    def tanh(self):
        x = 2 * self
        t = (x.exp() - 1) / (x.exp() + 1)
        out = Value(t.data, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t.data**2) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [16]:
# inputs: x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights: w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1; x1w1.label = 'x1w1'
x2w2 = x2 * w2; x2w2.label = 'x2w2'
b = Value(6.8813735870195, label='b')
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
o.backward()

In [20]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0

    def parameters(self):
        return []
        
class Neuron(Module):
    def __init__(self, n_ins):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(n_ins)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # forward pass: w * x + b
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer(Module):
    def __init__(self, n_in, n_out):
        self.neurons = [Neuron(n_in) for _ in range(n_out)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [param for neuron in self.neurons for param in neuron.parameters() ]

class MLP(Module):
    def __init__(self, n_in, n_outs):
        sz = [n_in] + n_outs
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(n_outs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [param for layer in self.layers for param in layer.parameters()]

In [21]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]
n = MLP(3, [4, 4, 1])

In [25]:
for i in range(500):
    ypreds = [n(x) for x in xs]
    loss = sum([(ypred - ytgt)**2 for ypred, ytgt in zip(ypreds, ys)])
    print("Loss: ", loss)
    n.zero_grad()
    loss.backward()
    for p in n.parameters():
        p.data += (-0.01) * p.grad

Loss:  Value(data=0.05180157767988157)
Loss:  Value(data=0.0510355889857491)
Loss:  Value(data=0.05029071653349781)
Loss:  Value(data=0.049566124671845194)
Loss:  Value(data=0.04886102036863703)
Loss:  Value(data=0.04817465057098486)
Loss:  Value(data=0.047506299756658515)
Loss:  Value(data=0.04685528766094146)
Loss:  Value(data=0.04622096716461704)
Loss:  Value(data=0.04560272233006281)
Loss:  Value(data=0.04499996657360627)
Loss:  Value(data=0.044412140963356085)
Loss:  Value(data=0.043838712632679605)
Loss:  Value(data=0.04327917330035837)
Loss:  Value(data=0.0427330378892311)
Loss:  Value(data=0.042199843235840424)
Loss:  Value(data=0.04167914688423488)
Loss:  Value(data=0.04117052595765496)
Loss:  Value(data=0.040673576102358285)
Loss:  Value(data=0.040187910498309484)
Loss:  Value(data=0.039713158931898154)
Loss:  Value(data=0.039248966926235855)
Loss:  Value(data=0.03879499492494409)
Loss:  Value(data=0.03835091752567015)
Loss:  Value(data=0.037916422759862504)
Loss:  Value(data

In [29]:
print("ypreds: ", ypreds)
print("target: ", ys)

ypreds:  [Value(data=0.9597082764732735), Value(data=-0.9653405392301367), Value(data=-0.9551970583491358), Value(data=0.9737649483122173)]
target:  [1.0, -1.0, -1.0, 1.0]
